<a href="https://colab.research.google.com/github/namvux1404/Anomalies-detection---Vector-workshop/blob/Nam/testrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/d-ailin/GDN.git

Cloning into 'GDN'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 54 (delta 10), reused 6 (delta 6), pack-reused 27
Receiving objects: 100% (54/54), 473.31 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
!pwd

/content


In [ ]:
!git clone https://github.com/namvux1404/Anomalies-detection---Vector-workshop.git

Cloning into 'Anomalies-detection---Vector-workshop'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 8.84 KiB | 8.84 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
#modify graph_layer.py
%cp /content/Anomalies-detection---Vector-workshop/modified_pyFile/graph_layer.py /content/GDN/models/graph_layer.py

In [ ]:
import os
os.chdir('./GDN/')

In [ ]:
import torch
print(torch.__version__)

2.0.1+cu118


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=fc8367129fd0a98430d1449167f91307c031efa2b2cde168d6033bc5b8db38b2
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 64.8 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Tue Aug 22 22:09:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!bash run.sh 0 msl

/content/GDN/models/GDN.py:161: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  gated_i = torch.arange(0, node_num).T.unsqueeze(1).repeat(1, topk_num).flatten().to(device).unsqueeze(0)
epoch (0 / 30) (Loss:0.46357875, ACU_loss:18.07957143)
epoch (1 / 30) (Loss:0.27831620, ACU_loss:10.85433182)
epoch (2 / 30) (Loss:0.23245400, ACU_loss:9.06570582)
epoch (3 / 30) (Loss:0.21226052, ACU_loss:8.27816018)
epoch (4 / 30) (Loss:0.19121092, ACU_loss:7.45722575)
epoch (5 / 30) (Loss:0.17493827, ACU_loss:6.82259235)
epoch (6 / 30) (Loss:0.17221259, ACU_loss:6.71629117)
epoch (7 / 30) (Loss:0.16269105, ACU_loss:6.34495105)
epoch (8 / 30) (Loss:0.14979067, ACU_loss:5.84183618)
epoch (9 / 

In [ ]:
!bash run.sh 0 msl

/content/GDN/models/GDN.py:161: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  gated_i = torch.arange(0, node_num).T.unsqueeze(1).repeat(1, topk_num).flatten().to(device).unsqueeze(0)
epoch (0 / 30) (Loss:0.46357875, ACU_loss:18.07957137)
epoch (1 / 30) (Loss:0.27831617, ACU_loss:10.85433082)
epoch (2 / 30) (Loss:0.23245515, ACU_loss:9.06575084)
epoch (3 / 30) (Loss:0.21231870, ACU_loss:8.28042929)
epoch (4 / 30) (Loss:0.19132835, ACU_loss:7.46180573)
epoch (5 / 30) (Loss:0.17451125, ACU_loss:6.80593866)
epoch (6 / 30) (Loss:0.17282181, ACU_loss:6.74005058)
epoch (7 / 30) (Loss:0.16399366, ACU_loss:6.39575265)
epoch (8 / 30) (Loss:0.15099483, ACU_loss:5.88879843)
epoch (9 / 

In [ ]:
#### try to update git

In [ ]:
#### Push to nam branch

In [1]:
#### blablalbal